In [163]:
# Import library and my api

import google.generativeai as genai
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

api_key = os.getenv("GPT_API_KEY")
google_key = os.getenv("GOOGLE_API_KEY")

In [ ]:
# Bring data from kaggle

import kagglehub

path = kagglehub.dataset_download("financial-news")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\RMARKET\.cache\kagglehub\datasets\notlucasp\financial-news-headlines\versions\2


In [165]:
import pandas as pd

df = pd.read_csv("cnbc_headlines.csv")

In [166]:
# Check data

df.head()

,Headlines,Time,Description
0,Jim Cramer: A better way to invest in the Covi...,"7:51 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer recommended buying..."
1,Cramer's lightning round: I would own Teradyne,"7:33 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer rings the lightnin..."
2,NaN,NaN,NaN
3,"Cramer's week ahead: Big week for earnings, ev...","7:25 PM ET Fri, 17 July 2020","""We'll pay more for the earnings of the non-Co..."
4,IQ Capital CEO Keith Bliss says tech and healt...,"4:24 PM ET Fri, 17 July 2020","Keith Bliss, IQ Capital CEO, joins ""Closing Be..."


In [167]:
print(df.shape)

(3080, 3)


In [168]:
df.drop (columns = 'Time', inplace = True)

In [169]:
df.head()

,Headlines,Description
0,Jim Cramer: A better way to invest in the Covi...,"""Mad Money"" host Jim Cramer recommended buying..."
1,Cramer's lightning round: I would own Teradyne,"""Mad Money"" host Jim Cramer rings the lightnin..."
2,NaN,NaN
3,"Cramer's week ahead: Big week for earnings, ev...","""We'll pay more for the earnings of the non-Co..."
4,IQ Capital CEO Keith Bliss says tech and healt...,"Keith Bliss, IQ Capital CEO, joins ""Closing Be..."


In [170]:
# Check number of null data

df.isnull().sum()

Headlines      280
Description    280
dtype: int64

In [171]:
# Drop these null data

df.dropna(inplace=True)

In [172]:
df.shape

(2800, 2)

In [173]:
print (3080 - 2800)

# If a null value appears, it means both the headline and description are null!
# Dropping it won't cause any data loss issues!

280


In [174]:
df.head()

,Headlines,Description
0,Jim Cramer: A better way to invest in the Covi...,"""Mad Money"" host Jim Cramer recommended buying..."
1,Cramer's lightning round: I would own Teradyne,"""Mad Money"" host Jim Cramer rings the lightnin..."
3,"Cramer's week ahead: Big week for earnings, ev...","""We'll pay more for the earnings of the non-Co..."
4,IQ Capital CEO Keith Bliss says tech and healt...,"Keith Bliss, IQ Capital CEO, joins ""Closing Be..."
5,Wall Street delivered the 'kind of pullback I'...,"""Look for the stocks of high-quality companies..."


In [175]:
# Combine Headline and Description

df ['news'] = df['Headlines'] + " <Description:>" +  df['Description']

df['news'].head()

0    Jim Cramer: A better way to invest in the Covi...
1    Cramer's lightning round: I would own Teradyne...
3    Cramer's week ahead: Big week for earnings, ev...
4    IQ Capital CEO Keith Bliss says tech and healt...
5    Wall Street delivered the 'kind of pullback I'...
Name: news, dtype: object

In [176]:
df.head()

,Headlines,Description,news
0,Jim Cramer: A better way to invest in the Covi...,"""Mad Money"" host Jim Cramer recommended buying...",Jim Cramer: A better way to invest in the Covi...
1,Cramer's lightning round: I would own Teradyne,"""Mad Money"" host Jim Cramer rings the lightnin...",Cramer's lightning round: I would own Teradyne...
3,"Cramer's week ahead: Big week for earnings, ev...","""We'll pay more for the earnings of the non-Co...","Cramer's week ahead: Big week for earnings, ev..."
4,IQ Capital CEO Keith Bliss says tech and healt...,"Keith Bliss, IQ Capital CEO, joins ""Closing Be...",IQ Capital CEO Keith Bliss says tech and healt...
5,Wall Street delivered the 'kind of pullback I'...,"""Look for the stocks of high-quality companies...",Wall Street delivered the 'kind of pullback I'...


In [178]:
# Make it into a list

headline_list = df['news'].to_list()
print(len(headline_list))
headline_list

2800


['Jim Cramer: A better way to invest in the Covid-19 vaccine gold rush <Description:>"Mad Money" host Jim Cramer recommended buying four companies that are supporting vaccine developers.',
 'Cramer\'s lightning round: I would own Teradyne <Description:>"Mad Money" host Jim Cramer rings the lightning round bell, which means he\'s giving his answers to callers\' stock questions at rapid speed.',
 'Cramer\'s week ahead: Big week for earnings, even bigger week for vaccines <Description:>"We\'ll pay more for the earnings of the non-Covid companies if The Lancet publishes some good news from AstraZeneca\'s vaccine trial," Cramer said.',
 'IQ Capital CEO Keith Bliss says tech and healthcare will rally <Description:>Keith Bliss, IQ Capital CEO, joins "Closing Bell" to talk about the broader markets, including the performance of the S&P 500 and the Nasdaq.',
 'Wall Street delivered the \'kind of pullback I\'ve been waiting for,\' Jim Cramer says <Description:>"Look for the stocks of high-qualit

In [350]:
# Choose this specific data (reason is shown on the ppt)

headline_list[28]

'The S&P 500 \'could get ugly\' in the near future, Jim Cramer warns <Description:>The S&P 500 could be riding into a "make-or-break moment" in the short term, the "Mad Money" host said.'

---

# chagpt-4o

In [321]:
os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI()

In [322]:
def comment_summarization(prompt, model="gpt-4o"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message.content

In [ ]:
response_summary = []

prompt = f"""
Your task is to analyze the 28th content of the list I give you. The input I gave you is in list format in Python. 
Only give one out of the following sentiments: [positive, negative, neutral]. 

Provide a brief summary of the news you analyze, then explain why you made your sentiment choice based on the index I give you. 
Ignore everything else in the list.

Text to analyze is this: ```{headline_list[28]}``` 
"""

# call the API to perform the task in the prompt
response = comment_summarization(prompt)
print(response, "\n")


**Sentiment:** Negative

**Summary:** Jim Cramer warns that the S&P 500 might face a challenging period soon, describing it as a "make-or-break moment" in the short term.

**Explanation:** The sentiment is negative because the language used by Jim Cramer, such as "could get ugly" and "make-or-break moment," suggests potential difficulties or downturns for the S&P 500 in the near future. These phrases indicate concern and caution, which are typically associated with negative sentiment. 



In [343]:
genai.configure(api_key=google_key)
model = genai.GenerativeModel('gemini-1.5-flash') 

In [ ]:
# gpt-4o

tokens = model.count_tokens(""" 
**Sentiment:** Negative
**Summary:** Jim Cramer warns that the S&P 500 might face a challenging period soon, describing it as a "make-or-break moment" in the short term.
**Explanation:** The sentiment is negative because the language used by Jim Cramer, such as "could get ugly" and "make-or-break moment," suggests potential difficulties or downturns for the S&P 500 in the near future. These phrases indicate concern and caution, which are typically associated with negative sentiment. 
""")
print(tokens)

total_tokens: 115



In [ ]:
# Checking if model remembers name.

# Translation: My name is James, What is my name -> your name is James

user_queries = [
    {'role': 'user', 'content': prompt},
]

history = []


for user_query in user_queries:
    history.append({"role": user_query["role"], "content": user_query["content"]})
    print(f'[사용자]: {user_query["content"]}')

    response = client.chat.completions.create(
        model="gpt-4o",  
        messages=history
    )
    
    model_response = response.choices[0].message.content
    
    print(f'[모델]: {model_response}')
    
    history.append({"role": "assistant", "content": model_response})

[사용자]: 내 이름은 제임스.
[모델]: 안녕하세요, 제임스! 만나서 반갑습니다. 어떻게 도와드릴까요?
[사용자]: 나의 이름은?
[모델]: 당신의 이름은 제임스입니다. 어떻게 더 도와드릴까요?


In [324]:
prompt = f"""
Your task is to analyze the 28th content of the list I give you. The input I gave you is in list format in Python. 
Only give one out of the following sentiments: [positive, negative, neutral]. 

Provide a brief summary of the news you analyze, then explain why you made your sentiment choice based on the index I give you. 
Ignore everything else in the list.

Text to analyze is this: ```{headline_list[28]}``` 
"""

user_queries = [
    {'role': 'user', 'content': prompt},
]

history = []

for user_query in user_queries:
    history.append({"role": user_query["role"], "content": user_query["content"]})
    print(f'[사용자]: {user_query["content"]}')

    response = client.chat.completions.create(
        model="gpt-4o",  
        messages=history
    )
    
    model_response = response.choices[0].message.content
    
    print(f'[모델]: {model_response}')
    
    history.append({"role": "assistant", "content": model_response})

while True:
    user_input = input("[사용자]: ")
    if user_input == "quit":
        print("대화 종료.")
        break
    else:
        history.append({"role": "user", "content": user_input})
        print(f'[사용자]: {user_input}')
        
        response = client.chat.completions.create(
            model="gpt-4",
            messages=history
        )
        
        model_response = response.choices[0].message.content
        print(f'[모델]: {model_response}')
        
        history.append({"role": "assistant", "content": model_response})

[사용자]: 
Your task is to analyze the 28th content of the list I give you. The input I gave you is in list format in Python. 
Only give one out of the following sentiments: [positive, negative, neutral]. 

Provide a brief summary of the news you analyze, then explain why you made your sentiment choice based on the index I give you. 
Ignore everything else in the list.

Text to analyze is this: ```The S&P 500 'could get ugly' in the near future, Jim Cramer warns <Description:>The S&P 500 could be riding into a "make-or-break moment" in the short term, the "Mad Money" host said.``` 

[모델]: **Sentiment:** Negative

**Summary:** Jim Cramer warns that the S&P 500 might face a challenging period, describing it as heading towards a "make-or-break moment" in the near future.

**Explanation:** The sentiment is negative because the statement by Jim Cramer suggests potential difficulties or downturns for the S&P 500, using phrases like "could get ugly" and "make-or-break moment," which indicate con

In [345]:
tokens = model.count_tokens(""" 
Sentiment: Negative
Summary: Jim Cramer warns of a potential negative turn for the S&P 500, as he predicts a "make-or-break moment" in the near future.
Explanation: The terms "could get ugly" and "make-or-break moment" suggest forthcoming adversity and uncertainty, indicating a negative sentiment.
""")
print(tokens)

total_tokens: 74



In [270]:
history = []

while True:
    index = int(input("index: "))
    
    prompt = f"""
    Your task is to analyze the content of the list I give you. The input I gave you is in list format in Python. 
    Only give one out of the following sentiments: [positive, negative, neutral]. 

    Provide a brief summary of the news you analyze, then explain why you made your sentiment choice based only on the item of the list. 
    Ignore everything else in the list.

    Text to analyze is this: ```{headline_list[index]}```
    """
    
    print(f"Prompt to analyze: {prompt}")
    
    history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model="gpt-4o",  
        messages=[{"role": "system", "content": "You are a helpful assistant."}] + [{"role": "user", "content": prompt}] + history,
        max_tokens=150  
    )

    model_output = response.choices[0].message.content 
    
    print(f'[모델]: {model_output}')
    history.append({"role": "assistant", "content": model_output})

    ans = input("[사용자]: ")
    print(f'[사용자]: {ans}')
    
    if ans.lower() == "quit":
        print("대화 종료.")
        break

Prompt to analyze: 
    Your task is to analyze the content of the list I give you. The input I gave you is in list format in Python. 
    Only give one out of the following sentiments: [positive, negative, neutral]. 

    Provide a brief summary of the news you analyze, then explain why you made your sentiment choice based only on the item of the list. 
    Ignore everything else in the list.

    Text to analyze is this: ```Cramer's lightning round: I would just stay long Wex <Description:>"Mad Money" host Jim Cramer rings the lightning round bell, which means he's giving his answers to callers' stock questions at rapid speed.```
    
[모델]: Summary: The text is about Jim Cramer, the host of "Mad Money," giving rapid-fire advice on stocks to callers. Specifically, it mentions his suggestion to "stay long" on the stock Wex during his lightning round segment.

Sentiment: Positive

Explanation: The phrase "I would just stay long Wex" implies a positive sentiment towards the stock, sugges

---

gpt4o-mini

In [325]:
def comment_summarization(prompt, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message.content

In [ ]:
response_summary = []

prompt = f"""
Your task is to analyze the 28th content of the list I give you. The input I gave you is in list format in Python. 
Only give one out of the following sentiments: [positive, negative, neutral]. 

Provide a brief summary of the news you analyze, then explain why you made your sentiment choice based on the index I give you. 
Ignore everything else in the list.

Text to analyze is this: ```{headline_list[28]}``` 
"""

response = comment_summarization(prompt)
print(response, "\n")


**Summary:** Jim Cramer, the host of "Mad Money," has expressed concerns about the S&P 500, indicating that it may face significant challenges in the near future. He describes the current situation as a "make-or-break moment," suggesting that the market could experience a downturn.

**Sentiment Choice:** Negative

**Explanation:** The phrase "could get ugly" and the description of the market being at a "make-or-break moment" imply a warning about potential declines or difficulties ahead. This language conveys a sense of urgency and concern, leading to a negative sentiment regarding the future performance of the S&P 500. 



In [346]:
tokens = model.count_tokens(""" 
**Summary:** Jim Cramer, the host of "Mad Money," has expressed concerns about the S&P 500, indicating that it may face significant challenges in the near future. He describes the current situation as a "make-or-break moment," suggesting that the market could experience a downturn.
**Sentiment Choice:** Negative
**Explanation:** The phrase "could get ugly" and the description of the market being at a "make-or-break moment" imply a warning about potential declines or difficulties ahead. This language conveys a sense of urgency and concern, leading to a negative sentiment regarding the future performance of the S&P 500.
""")
print(tokens)

total_tokens: 138



In [327]:
user_queries = [
    {'role': 'user', 'content': prompt},
]

history = []


for user_query in user_queries:
    history.append({"role": user_query["role"], "content": user_query["content"]})
    print(f'[사용자]: {user_query["content"]}')

    response = client.chat.completions.create(
        model="gpt-4o-mini",  
        messages=history
    )
    
    model_response = response.choices[0].message.content
    
    print(f'[모델]: {model_response}')
    
    history.append({"role": "assistant", "content": model_response})

[사용자]: 
Your task is to analyze the 28th content of the list I give you. The input I gave you is in list format in Python. 
Only give one out of the following sentiments: [positive, negative, neutral]. 

Provide a brief summary of the news you analyze, then explain why you made your sentiment choice based on the index I give you. 
Ignore everything else in the list.

Text to analyze is this: ```The S&P 500 'could get ugly' in the near future, Jim Cramer warns <Description:>The S&P 500 could be riding into a "make-or-break moment" in the short term, the "Mad Money" host said.``` 

[모델]: **Summary:** The news discusses a warning from Jim Cramer regarding the S&P 500 index, suggesting that it may face significant challenges in the near future and could enter a critical phase.

**Sentiment Choice:** Negative

**Explanation:** The use of phrases like "could get ugly" and "make-or-break moment" conveys a sense of impending trouble or instability for the S&P 500, indicating a pessimistic outl

In [ ]:
history = []

while True:
    index = int(input("index: "))
    
    prompt = f"""
    Your task is to analyze the content of the list I give you. The input I gave you is in list format in Python. 
    Only give one out of the following sentiments: [positive, negative, neutral]. 

    Provide a brief summary of the news you analyze, then explain why you made your sentiment choice based only on the item of the list. 
    Ignore everything else in the list.

    Text to analyze is this: ```{headline_list[index]}```
    """
    
    print(f"Prompt to analyze: {prompt}")
    
    history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model="gpt-4o-mini",  
        messages=[{"role": "system", "content": "You are a helpful assistant."}] + [{"role": "user", "content": prompt}] + history,
        max_tokens=150  
    )

    model_output = response.choices[0].message.content 
    
    print(f'[모델]: {model_output}')
    history.append({"role": "assistant", "content": model_output})

    ans = input("[사용자]: ")
    print(f'[사용자]: {ans}')
    
    if ans.lower() == "quit":
        print("대화 종료.")
        break

Prompt to analyze: 
    Your task is to analyze the content of the list I give you. The input I gave you is in list format in Python. 
    Only give one out of the following sentiments: [positive, negative, neutral]. 

    Provide a brief summary of the news you analyze, then explain why you made your sentiment choice based only on the item of the list. 
    Ignore everything else in the list.

    Text to analyze is this: ```The S&P 500 'could get ugly' in the near future, Jim Cramer warns <Description:>The S&P 500 could be riding into a "make-or-break moment" in the short term, the "Mad Money" host said.```
    
[모델]: Summary: The news item discusses Jim Cramer's warning about the S&P 500, indicating that the stock index could face a challenging period soon. He describes the situation as a potential "make-or-break moment" in the short term.

Sentiment: Negative

Explanation: The sentiment is negative because the language used, such as "could get ugly" and "make-or-break moment," indi

In [347]:
tokens = model.count_tokens(""" 
Summary: The news item features Jim Cramer's warning regarding the S&P 500, suggesting that the index might face significant difficulties or turbulence soon. He refers to this upcoming period as a possible "make-or-break moment."
Sentiment Choice: Negative
Explanation: The sentiment is negative because the phrases "could get ugly" and "make-or-break moment" imply an expectation of potential risk or adverse developments for the S&P 500. This language conveys a sense of caution and concern about the market's near-term outlook.
""")
print(tokens)

total_tokens: 120



In [ ]:
# 기억함

prompt = f"""
Your task is to analyze the 28th content of the list I give you. The input I gave you is in list format in Python. 
Only give one out of the following sentiments: [positive, negative, neutral]. 

Provide a brief summary of the news you analyze, then explain why you made your sentiment choice based on the index I give you. 
Ignore everything else in the list.

Text to analyze is this: ```{headline_list[28]}``` 
"""

user_queries = [
    {'role': 'user', 'content': prompt},
]

history = []

for user_query in user_queries:
    history.append({"role": user_query["role"], "content": user_query["content"]})
    print(f'[사용자]: {user_query["content"]}')

    response = client.chat.completions.create(
        model="gpt-4o",  
        messages=history
    )
    
    model_response = response.choices[0].message.content
    
    print(f'[모델]: {model_response}')
    
    history.append({"role": "assistant", "content": model_response})

while True:
    user_input = input("[사용자]: ")
    if user_input == "quit":
        print("대화 종료.")
        break
    else:
        history.append({"role": "user", "content": user_input})
        print(f'[사용자]: {user_input}')
        
        response = client.chat.completions.create(
            model="gpt-4",
            messages=history
        )
        
        model_response = response.choices[0].message.content
        print(f'[모델]: {model_response}')
        
        history.append({"role": "assistant", "content": model_response})

---

In [330]:
genai.configure(api_key=google_key)
model = genai.GenerativeModel('gemini-1.5-flash') 

In [331]:
response = model.generate_content(f"""
Your task is to analyze the 28th content of the list I give you. The input I gave you is in list format in Python. 
Only give one out of the following sentiments: [positive, negative, neutral]. 

Provide a brief summary of the news you analyze, then explain why you made your sentiment choice based on the index I give you. 
Ignore everything else in the list.

Text to analyze is this: ```{headline_list[28]}``` 
""")

print (response.text)

**Summary:**  Jim Cramer, host of "Mad Money," warns that the S&P 500 might experience a significant downturn ("get ugly") in the near future, characterizing the current situation as a critical juncture.

**Sentiment:** Negative

**Reasoning:** The phrase "could get ugly" clearly indicates a negative outlook for the S&P 500.  The use of the word "warns" further reinforces the negative sentiment, suggesting an impending negative event. The description of the situation as a "make-or-break moment" also implies a high risk of negative consequences.



In [348]:
tokens = model.count_tokens(""" 
**Summary:**  Jim Cramer, host of "Mad Money," warns that the S&P 500 might experience a significant downturn ("get ugly") in the near future, characterizing the current situation as a critical juncture.
**Sentiment:** Negative
**Reasoning:** The phrase "could get ugly" clearly indicates a negative outlook for the S&P 500.  The use of the word "warns" further reinforces the negative sentiment, suggesting an impending negative event. The description of the situation as a "make-or-break moment" also implies a high risk of negative consequences.
""")
print(tokens)

total_tokens: 127



In [ ]:
# Define the prompt correctly in a dictionary
prompt = f"""
Your task is to analyze the 28th content of the list I give you. The input I gave you is in list format in Python. 
Only give one out of the following sentiments: [positive, negative, neutral]. 

Provide a brief summary of the news you analyze, then explain why you made your sentiment choice based on the index I give you. 
Ignore everything else in the list.

Text to analyze is this: ```{headline_list[28]}``` 
"""

user_query = {'role': 'user', 'parts': prompt}

# Initialize history list
history = []

history.append(user_query)

print(f'[사용자]: {user_query["parts"]}') 

response = model.generate_content(history)

print(f'[모델]: {response.text}')   

history.append(response.candidates[0].content)

while True:
    ans = input("[사용자]: ")
    if ans == "quit":
        print("대화 종료.")
        break
    
    history.append({"role": "user", "parts": ans})
    print(f'[사용자]: {ans}')
    
    response = model.generate_content(history)
    print(f'[모델]: {response.text}')  
    history.append(response.candidates[0].content)


[사용자]: 
Your task is to analyze the 28th content of the list I give you. The input I gave you is in list format in Python. 
Only give one out of the following sentiments: [positive, negative, neutral]. 

Provide a brief summary of the news you analyze, then explain why you made your sentiment choice based on the index I give you. 
Ignore everything else in the list.

Text to analyze is this: ```The S&P 500 'could get ugly' in the near future, Jim Cramer warns <Description:>The S&P 500 could be riding into a "make-or-break moment" in the short term, the "Mad Money" host said.``` 

[모델]: **Summary:**  Jim Cramer, host of "Mad Money," warns that the S&P 500 might experience a significant downturn ("get ugly") in the near future, characterizing the current situation as a critical juncture.

**Sentiment:** Negative

**Reasoning:** The phrase "could get ugly" is inherently negative, suggesting a potential for substantial losses or a significant market decline.  The use of "make-or-break mome

In [342]:
history = []

while True:
    index = int(input("index: "))
    prompt = f"""
    Your task is to analyze the content of the list I give you. The input I gave you is in list format in Python. 
    Only give one out of the following sentiments: [positive, negative, neutral]. 

    Provide a brief summary of the news you analyze, then explain why you made your sentiment choice based only on the item of the list. 
    Ignore everything else in the list.

    Text to analyze is this: ```{headline_list[index]}```
    """
    
    print(f"Prompt to analyze: {prompt}")
    
    history.append({"role": "user", "parts": prompt})
    
    response = model.generate_content(history)
    print(f'[모델]: {response.text}')  
    history.append(response.candidates[0].content)    

    ans = input("[사용자]: ")
    print(f'[사용자]: {ans}')
    
    if ans == "quit":
        print("대화 종료.")
        break

Prompt to analyze: 
    Your task is to analyze the content of the list I give you. The input I gave you is in list format in Python. 
    Only give one out of the following sentiments: [positive, negative, neutral]. 

    Provide a brief summary of the news you analyze, then explain why you made your sentiment choice based only on the item of the list. 
    Ignore everything else in the list.

    Text to analyze is this: ```The S&P 500 'could get ugly' in the near future, Jim Cramer warns <Description:>The S&P 500 could be riding into a "make-or-break moment" in the short term, the "Mad Money" host said.```
    
[모델]: **Sentiment:** Negative

**Summary:**  Jim Cramer, host of "Mad Money," warns that the S&P 500 could experience a significant downturn ("get ugly") in the near future, characterizing the situation as a critical "make-or-break moment."

**Explanation:** The phrase "could get ugly" is inherently negative, suggesting a potential for substantial losses or a negative market 

In [ ]:
tokens = model.count_tokens(""" 
**Explanation:** The phrase "could get ugly" is strongly suggestive of a negative market event.  Coupled with the description of a "make-or-break moment," the overall tone is pessimistic, forecasting significant risk and potential for substantial losses in the S&P 500.  The source, a known financial commentator, adds weight to the negative sentiment.
""")
print(tokens)

# Why cut off too much!!!

total_tokens: 78



---